## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-22-05-46-40 +0000,bbc,UK warns Israel not to retaliate against Pales...,https://www.bbc.com/news/articles/c1wggrdn9dno...
1,2025-09-22-05-16-50 +0000,nypost,Kamala Harris reveals what Biden told her just...,https://nypost.com/2025/09/22/us-news/kamala-h...
2,2025-09-22-04-45-35 +0000,bbc,Outdoor brand Arc'teryx apologises for firewor...,https://www.bbc.com/news/articles/c1eddw19042o...
3,2025-09-22-04-44-08 +0000,nypost,"Hometown vigil held for Celeste Rivas, 15-year...",https://nypost.com/2025/09/22/us-news/vigil-he...
4,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,40
48,kirk,27
47,charlie,21
201,memorial,12
5,palestinian,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
4,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...,123
22,2025-09-22-02-09-32 +0000,bbc,Trump hails Charlie Kirk as 'American hero' as...,https://www.bbc.com/news/articles/ckgee0x9p40o...,119
59,2025-09-21-22-51-23 +0000,nypost,Trump calls Charlie Kirk one of ‘America’s gre...,https://nypost.com/2025/09/21/us-news/trump-ca...,118
50,2025-09-21-23-30-00 +0000,wsj,President Trump and the upper echelon of his a...,https://www.wsj.com/us-news/charlie-kirk-funer...,117
71,2025-09-21-21-50-25 +0000,nypost,"President Trump, Elon Musk shake hands at Char...",https://nypost.com/2025/09/21/us-news/presiden...,115


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
4,123,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...
141,53,2025-09-21-13-36-12 +0000,nypost,"Canada, UK and Australia all recognize Palesti...",https://nypost.com/2025/09/21/world-news/canad...
61,48,2025-09-21-22-27-41 +0000,nypost,Erika Kirk forgives assassin who killed husban...,https://nypost.com/2025/09/21/us-news/erika-ki...
15,32,2025-09-22-03-21-19 +0000,nypost,New dad receives 24-year-old letter penned by ...,https://nypost.com/2025/09/21/us-news/new-dad-...
96,30,2025-09-21-20-00-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
71,27,2025-09-21-21-50-25 +0000,nypost,"President Trump, Elon Musk shake hands at Char...",https://nypost.com/2025/09/21/us-news/presiden...
59,26,2025-09-21-22-51-23 +0000,nypost,Trump calls Charlie Kirk one of ‘America’s gre...,https://nypost.com/2025/09/21/us-news/trump-ca...
75,25,2025-09-21-21-22-56 +0000,latimes,Fan gives back Mike Trout's 400th career home ...,https://www.latimes.com/sports/story/2025-09-2...
21,23,2025-09-22-02-26-46 +0000,nypost,"Innocent teen girl shot, man stabbed in separa...",https://nypost.com/2025/09/21/us-news/teen-bys...
129,23,2025-09-21-15-04-14 +0000,nypost,Jasmine Crockett scolds white Dems for not vot...,https://nypost.com/2025/09/21/us-news/jasmine-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
